In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

from stanley_appex.estimation import *
from stanley_appex.plotting import *
from stanley_appex.utils import *
from stanley_appex.generate_data import *
import numpy as np
import matplotlib.pyplot as plt
from yian_grit.GRIT_src import *
import json
from tqdm import tqdm

In [3]:
print(os.getcwd())
files = [os.path.join('data/comparison', folder) for folder in os.listdir('data/comparison')]
print(len(files))

/Users/stanleynicholson/code/Bjorn/Optimal Transport/OTProject/trajectory_inference/src
60


In [4]:
appex_done = os.path.exists('grit_vs_appex/appex_results.json')
print('Appex done:', appex_done)
grit_done = os.path.exists('grit_vs_appex/grit_results.json')
print('GRIT done:', grit_done)

Appex done: True
GRIT done: False


In [5]:
process = BranchingStochasticProcess()
process.load_data(files[0])
# process.run_appex(downsample_rate=1000)
# plt.plot(process.A_error)

15000


<HDF5 file "65c5f10c-2781-11f0-a01d-5e4e2ac260d2.h5" (mode r)>

In [13]:
noise_strengths = [0.05, 0.1, 0.5, 1.0]
dims = [2, 5, 10, 20]
maxiters = 10
downsample_rate = 50
N_sample = 2*process.N_traj
A_errors = np.zeros((len(noise_strengths), len(dims), maxiters + 2))
H_errors = np.zeros((len(noise_strengths), len(dims), maxiters + 2))
results = {}

In [19]:
for file in tqdm(files):
    if file in results.keys():
        continue
    process = BranchingStochasticProcess()
    process.load_data(file)
    
    if not appex_done:
        process.run_appex(downsample_rate=downsample_rate, maxiters=maxiters, print_out=1, N_sample=N_sample)
        i = noise_strengths.index(process.G[0, 0])
        j = dims.index(process.d)
        A_errors[i, j, :] = process.A_error
        H_errors[i, j, :] = process.A_error
        np.save('grit_vs_appex/APPEX_A_errors.npy', A_errors)
        np.save('grit_vs_appex/APPEX_H_errors.npy', H_errors)
        results[file] = {
            'A_error': process.A_error[-1],
            'H_error': process.H_error[-1],
            'A': process.A,
            'G': process.G,
            'A_est': process.A_est,
            'H_est': process.H_est,
            'd': process.d,
            'noise_strength': process.G[0, 0],
            'maxiters': maxiters,
            'N_sample': N_sample,
            'N_traj': process.N_traj,
            'downsample_rate': downsample_rate
        }
    if not grit_done:
        downsample_xs_data = process.xs_data.transpose((1, 2, 0))[::downsample_rate]
        downsample_ts_data = process.ts_data[::downsample_rate]
        noise_strength = process.G[0, 0]
        grit_output = GRIT_MATLAB_No_adhock_nob(downsample_xs_data, downsample_ts_data, epsilon=noise_strength)
        A_grit = grit_output[0]
        A_error = np.linalg.norm(process.A - process.A, ord="fro")
        H_error = 0.0
        results[file] = {
            'A_error': A_error,
            'H_error': H_error,
            'A': process.A,
            'G': process.G,
            'A_est': A_grit,
            'H_est': noise_strength,
            'd': process.d,
            'noise_strength': noise_strength,
            'N_sample': N_sample,
            'N_traj': process.N_traj,
            'downsample_rate': downsample_rate,
            'algorithm': 'GRIT'
        }


  0%|          | 0/60 [00:00<?, ?it/s]

15000


  2%|▏         | 1/60 [00:43<42:25, 43.14s/it]

15000


  3%|▎         | 2/60 [01:18<37:22, 38.67s/it]

...

 97%|█████████▋| 58/60 [2:54:14<03:46, 113.29s/it]

15000


 98%|█████████▊| 59/60 [2:56:49<02:05, 125.81s/it]

15000


100%|██████████| 60/60 [3:02:50<00:00, 182.85s/it]


In [ ]:
if not os.path.exists('grit_vs_appex/APPEX_A_errors_final.npy'):
    np.save('grit_vs_appex/APPEX_A_errors_final.npy', A_errors)
    np.save('grit_vs_appex/APPEX_H_errors_final.npy', H_errors)

In [ ]:
def process_dict(results, file_path=None):
    rdict = {}
    for file, result in results.items():
        rdict[file] = {
            'A_error': result['A_error'].astype(float),
            'H_error': float(result['H_error']),
            'A': result['A'].tolist(),
            'G': result['G'].tolist(),
            'A_est': result['A_est'].tolist(),
            'H_est': result['H_est'].tolist(),
            'd': int(result['d']),
            'noise_strength': result['noise_strength'].astype(float),
            'maxiters': int(result['maxiters']),
            'N_sample': int(result['N_sample']),
            'N_traj': int(result['N_traj']),
            'downsample_rate': int(result['downsample_rate'])
        }
    if file_path is not None:
        with open(file_path, 'w') as f:
            json.dump(rdict, f)
    return rdict

In [25]:
if not appex_done:
    process_dict(results, 'grit_vs_appex/appex_results.json')
if not grit_done:
    process_dict(results, 'grit_vs_appex/grit_results.json')